In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# Load datasets
places_df = pd.read_csv("Top Indian Places to Visit.csv")
places_df.head(5)

In [ ]:
airlines_df = pd.read_csv("Indian Airlines.csv")
airlines_df.head(5)

In [53]:
print(airlines_df.columns)

Index(['Unnamed: 0', 'airline', 'flight', 'source_city', 'departure_time',
       'stops', 'arrival_time', 'destination_city', 'class', 'duration',
       'days_left', 'price'],
      dtype='object')


In [55]:
places_df['City'] = places_df['City'].str.lower()
airlines_df['destination_city'] = airlines_df['destination_city'].str.lower()  # Use correct column name

merged_df = places_df.merge(airlines_df, left_on='City', right_on='destination_city', how='inner')
merged_df.head

<bound method NDFrame.head of          Unnamed: 0_x      Zone      State       City                    Name  \
0                   0  Northern      Delhi      delhi              India Gate   
1                   0  Northern      Delhi      delhi              India Gate   
2                   0  Northern      Delhi      delhi              India Gate   
3                   0  Northern      Delhi      delhi              India Gate   
4                   0  Northern      Delhi      delhi              India Gate   
...               ...       ...        ...        ...                     ...   
2769759           312  Southern  Telangana  hyderabad  Inorbit Mall Cyberabad   
2769760           312  Southern  Telangana  hyderabad  Inorbit Mall Cyberabad   
2769761           312  Southern  Telangana  hyderabad  Inorbit Mall Cyberabad   
2769762           312  Southern  Telangana  hyderabad  Inorbit Mall Cyberabad   
2769763           312  Southern  Telangana  hyderabad  Inorbit Mall Cyberabad  

In [56]:
print(merged_df.columns)

Index(['Unnamed: 0_x', 'Zone', 'State', 'City', 'Name', 'Type',
       'Establishment Year', 'time needed to visit in hrs',
       'Google review rating', 'Entrance Fee in INR',
       'Airport with 50km Radius', 'Weekly Off', 'Significance',
       'DSLR Allowed', 'Number of google review in lakhs',
       'Best Time to visit', 'Unnamed: 0_y', 'airline', 'flight',
       'source_city', 'departure_time', 'stops', 'arrival_time',
       'destination_city', 'class', 'duration', 'days_left', 'price'],
      dtype='object')


In [75]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load datasets
places_df = pd.read_csv("Top Indian Places to Visit.csv")
airlines_df = pd.read_csv("Indian Airlines.csv")

# Data Preprocessing: Combine relevant text features for recommendation
places_df['combined_features'] = places_df['Type'] + " " + places_df['Significance'] + " " + places_df['Best Time to visit']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(places_df['combined_features'])

# Function to recommend places based on user preferences
def recommend_places(user_interest, preferred_time, budget=5000, top_n=5):
    user_input = user_interest + " " + preferred_time
    user_vector = vectorizer.transform([user_input])
    similarity_scores = cosine_similarity(user_vector, tfidf_matrix).flatten()
    top_indices = np.argsort(similarity_scores)[-top_n:][::-1]
    
    recommended_places = places_df.iloc[top_indices]
    recommended_places = recommended_places[recommended_places['Entrance Fee in INR'] <= budget]
    return recommended_places[['Name', 'City', 'State', 'Type', 'Google review rating']]

# Function to recommend flights based on user destination and budget
def recommend_flights(destination_city, budget=10000, top_n=3):
    flights = airlines_df[airlines_df['destination_city'] == destination]
    flights = flights[flights['price'] <= budget]
    return flights.sort_values(by='price').head(top_n)[['airline', 'source_city', 'destination_city', 'departure_time', 'arrival_time', 'price']]

# Example: Generate full itinerary for a user
user_interest = "historical sites"
preferred_time = "Morning"
budget = 5000
destination = "Ahmedabad"

recommended_places = recommend_places(user_interest, preferred_time, budget)
recommended_flights = recommend_flights(destination, budget)

# Display itinerary
print("Recommended Places:")
print(recommended_places)
print("\nRecommended Flights:")
print(recommended_flights)


Recommended Places:
                 Name       City           State        Type  \
66   Sabarmati Ashram  Ahmedabad         Gujarat  Historical   
256    Gandikota Fort     Kadapa  Andhra Pradesh        Fort   
139      Gwalior Fort    Gwalior  Madhya Pradesh        Fort   
54        Fort Aguada        Goa             Goa        Fort   
33      Golconda Fort  Hyderabad       Telangana        Fort   

     Google review rating  
66                    4.6  
256                   4.5  
139                   4.5  
54                    4.2  
33                    4.4  

Recommended Flights:
Empty DataFrame
Columns: [airline, source_city, destination_city, departure_time, arrival_time, price]
Index: []


In [77]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample dataset with destination names, descriptions, and reviews
data = {
    'destination': ['Paris', 'London', 'New York', 'Tokyo', 'Barcelona'],
    'description': [
        'The Eiffel Tower and other landmarks. Beautiful city with rich culture.',
        'Home of Big Ben and the British Museum. Great for history lovers.',
        'Known for Central Park, Times Square, and Statue of Liberty. A bustling city.',
        'Famous for Mount Fuji, beautiful temples, and cherry blossoms in spring.',
        'Known for Gaudi architecture, the Sagrada Familia, and beaches.'
    ],
    'reviews': [
        'Amazing view from the Eiffel Tower. Great food and culture.',
        'A historic city with so much to explore. Loved the museums.',
        'A city that never sleeps. Lots of shopping, eating, and entertainment.',
        'A peaceful city with nature and beautiful culture. A must-visit!',
        'Stunning architecture and beaches. Worth visiting for culture lovers.'
    ]
}

# Convert data into a pandas DataFrame
df = pd.DataFrame(data)

# Combine descriptions and reviews into one text field
df['text'] = df['description'] + " " + df['reviews']

# Step 1: Create a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Step 2: Fit and transform the combined text data to get the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Step 3: Calculate cosine similarity between the destinations
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 4: Function to get recommendations based on a destination name
def get_recommendations(destination, cosine_sim=cosine_sim):
    # Get the index of the destination
    idx = df[df['destination'] == destination].index[0]
    
    # Get the pairwise similarity scores for all destinations
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the destinations based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 3 recommendations (excluding the first one, which is the destination itself)
    sim_scores = sim_scores[1:4]
    
    # Get the destination names and similarity scores
    recommended_destinations = [df['destination'][i[0]] for i in sim_scores]
    
    return recommended_destinations

# Example: Get recommendations for 'Paris'
recommended = get_recommendations('Paris')
print("Recommended destinations for Paris:", recommended)


Recommended destinations for Paris: ['Tokyo', 'London', 'Barcelona']
